In [3]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import os
import numpy as np
import pandas as pd
from models.aa_common.data_loader import get_patho_and_likelypatho_SNVs

In [4]:
task = "patho_and_likelypatho"
variants_df = get_patho_and_likelypatho_SNVs(home_dir)


Log: Loading base-patho data ...
Index(['clinvar_id', 'gene_name', 'gene_id', 'snp_id', 'mrna_acc_version',
       'mrna_gi', 'prot_variant', 'prot_acc_version', '1indexed_prot_mt_pos',
       'wt_aa', 'mt_aa', 'wt_aa_1letter', 'mt_aa_1letter', 'chrom_variant',
       'chrom_acc_version', 'chrom_num', 'chrom_pos', 'ref_allele',
       'alt_allele', 'class'],
      dtype='object')
total:  (7332, 20)
#-unique_clinvarids: 7328
#-unique_rsids: 300
#-unique_genes: 1599
#-unique_NP_ids: 1603
Likely-pathogenic    4818
Pathogenic           2514
Name: class, dtype: int64


In [5]:
# merging dbNSFP extracted scores with popu-freq dataset.
pred_df = pd.read_csv(home_dir+f"models/dbnsfp/outputs_dbnsfp/{task}.txt", sep="\t")
print(pred_df.shape)
pred_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
print(pred_df.columns)
print(pred_df.shape)
pred_df

(7454, 16)
Index(['#chr', 'pos(1-based)', 'ref', 'alt', 'aaref', 'aaalt', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw', 'integrated_fitCons_score', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic'],
      dtype='object')
(7454, 16)


,#chr,pos(1-based),ref,alt,aaref,aaalt,SIFT_score,Polyphen2_HVAR_score,MetaRNN_score,REVEL_score,MVP_score,CADD_raw,integrated_fitCons_score,phyloP17way_primate,phastCons17way_primate,bStatistic
0,1,976215,A,G,V,A,0.024;.,.;.,7.7831055e-07;7.7831055e-07,0.016;0.016,.;.,1.476529,0.615755,0.688,0.633,878
1,1,1232280,T,C,M,T,.;0.0,0.026;0.026,0.97040886;0.97040886,.;0.203,.;0.498705051145,2.745302,0.437478,0.408,0.737,745
2,1,1232281,G,A,M,I,.;0.0,0.026;0.026,0.96377194;0.96377194,.;0.240,.;0.639788913552,2.915920,0.437478,0.411,0.754,745
3,1,1312290,C,T,V,M,0.006;0.007;.;.;0.007;0.005;.;0.004;0.007,.;.;.;.;0.459;0.428;.;0.632;0.66,0.59258085;0.59258085;0.59258085;0.59258085;0....,0.158;0.158;.;.;0.158;0.158;.;0.158;0.158,0.654906220855;0.654906220855;0.654906220855;0...,3.357409,0.706548,0.594,0.668,774
4,1,1312841,G,A,H,Y,0.0;0.0;.;0.0;0.0;.;0.0;0.0,.;.;.;1.0;1.0;.;1.0;1.0,0.9879951;0.9879951;0.9879951;0.9879951;0.9879...,0.874;0.874;.;0.874;0.874;.;0.874;0.874,0.945332695629;0.945332695629;0.945332695629;0...,3.542766,0.67177,0.676,0.612,774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7449,X,154765939,C,G,H,Q,.;0.001;0.001,.;0.4;.,0.9535749;0.9535749;0.9535749,.;0.792;0.792,0.994831105455;0.994831105455;0.994831105455,2.446405,.,-0.176,0.892,81
7450,X,154769246,G,A,R,Q,.;0.028;0.04,.;0.86;.,0.6445384;0.6445384;0.6445384,.;0.691;0.691,0.966832675518;0.966832675518;0.966832675518,4.223634,.,0.676,0.989,81
7451,X,154770817,A,G,D,G,.;0.046,.;0.439,0.6140989;0.6140989,.;0.696,0.971271616849;0.971271616849,3.673606,.,0.731,0.941,81
7452,Y,2787299,G,A,T,I,0.0,0.983,0.8814292,.,0.989895515135,2.317115,.,0.196,0.435,.


In [6]:
pred_df["#chr"] = pred_df.apply(lambda row: str(row["#chr"]), axis=1) # this line is mandatory

# computing average scores for each method for each row
def compute_avg(x):
    x = str(x).split(";")
    return np.mean([float(i) for i in x if i!="."]) 

model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score', 
               'integrated_fitCons_score', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic']
for model_name in model_names:
    model_scores = pred_df[model_name].apply(compute_avg) # can have multiple scores, ie '0.4573521;0.4573521;0.4573521;0.4573521'. taking the avg
    pred_df[model_name] = model_scores

print(f"#-of SNVs found from dbNSFP: {pred_df.shape[0]}")
pred_df

/projects/ashehu/akabir4/venvs/hopper_variant_effect_analysis_mine/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


#-of SNVs found from dbNSFP: 7454


,#chr,pos(1-based),ref,alt,aaref,aaalt,SIFT_score,Polyphen2_HVAR_score,MetaRNN_score,REVEL_score,MVP_score,CADD_raw,integrated_fitCons_score,phyloP17way_primate,phastCons17way_primate,bStatistic
0,1,976215,A,G,V,A,0.024,NaN,7.783105e-07,0.016,NaN,1.476529,0.615755,0.688,0.633,878.0
1,1,1232280,T,C,M,T,0.000,0.02600,9.704089e-01,0.203,0.498705,2.745302,0.437478,0.408,0.737,745.0
2,1,1232281,G,A,M,I,0.000,0.02600,9.637719e-01,0.240,0.639789,2.915920,0.437478,0.411,0.754,745.0
3,1,1312290,C,T,V,M,0.006,0.54475,5.925809e-01,0.158,0.654906,3.357409,0.706548,0.594,0.668,774.0
4,1,1312841,G,A,H,Y,0.000,1.00000,9.879951e-01,0.874,0.945333,3.542766,0.671770,0.676,0.612,774.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7449,X,154765939,C,G,H,Q,0.001,0.40000,9.535749e-01,0.792,0.994831,2.446405,NaN,-0.176,0.892,81.0
7450,X,154769246,G,A,R,Q,0.034,0.86000,6.445384e-01,0.691,0.966833,4.223634,NaN,0.676,0.989,81.0
7451,X,154770817,A,G,D,G,0.046,0.43900,6.140989e-01,0.696,0.971272,3.673606,NaN,0.731,0.941,81.0
7452,Y,2787299,G,A,T,I,0.000,0.98300,8.814292e-01,NaN,0.989896,2.317115,NaN,0.196,0.435,NaN


In [7]:
result_df = pd.merge(left=variants_df, right=pred_df, how='inner', 
                     left_on=['chrom_num', 'chrom_pos', 'ref_allele', 'alt_allele', 'wt_aa_1letter', 'mt_aa_1letter'], 
                     right_on=['#chr', 'pos(1-based)', 'ref', 'alt', 'aaref', 'aaalt'])
result_df.shape

(7303, 36)

In [8]:
cols = variants_df.columns.to_list() + model_names
result_df = result_df[cols]

In [9]:
result_df.to_csv(home_dir+f"models/dbnsfp/outputs_postprocessed/{task}.tsv", sep="\t", index=False)

In [ ]:
result_df = pd.read_csv(home_dir+f"models/dbnsfp/outputs_postprocessed/{task}.tsv", sep="\t")

In [14]:
def print_missing_things(x:pd.DataFrame, classes):
    print("\t", end="")
    for i, cls in enumerate(classes):
        prots = x[(x["class"]==cls)]["prot_acc_version"].unique().shape[0]
        print(f"{cls}({prots})", end="\t")
    print()
    for model_name in model_names:
        print(model_name, end="\t")
        for i, cls in enumerate(classes):
            missing = x[(x["class"]==cls) & pd.isna(x[model_name])].shape[0]
            not_missing = x[(x["class"]==cls) & ~pd.isna(x[model_name])].shape[0]
            total = x[(x["class"]==cls)].shape[0]
            
            print(f"{missing}/{not_missing}", end="\t")
            if i==len(classes)-1: print()

print_missing_things(result_df, ["Pathogenic", "Likely-pathogenic"])

	Pathogenic(967)	Likely-pathogenic(1210)	
MetaRNN_score	0/2499	0/4804	
MVP_score	13/2486	11/4793	
SIFT_score	82/2417	136/4668	
Polyphen2_HVAR_score	108/2391	211/4593	
CADD_raw	0/2499	0/4804	
REVEL_score	48/2451	68/4736	
integrated_fitCons_score	293/2206	527/4277	
phyloP17way_primate	0/2499	0/4804	
phastCons17way_primate	0/2499	0/4804	
bStatistic	44/2455	74/4730	


In [11]:

def print_summary(df, classes):
    print("", "#-genes", "#-proteins", "#-protein-variants", "#-unique-genomic-variants", sep="\t")
    for cls in classes:
        cls_df = df[df["class"]==cls]
        n_genes = cls_df["gene_name"].unique().shape[0]
        n_prots = cls_df["prot_acc_version"].unique().shape[0]
        n_prot_variants = cls_df["prot_variant"].shape[0] # this and the following line are same
        n_genomic_variants = cls_df[["chrom_num", "chrom_pos", "ref_allele", "alt_allele"]].drop_duplicates(keep="first").shape[0] # these keywords are from ALFAs
        print(cls, n_genes, n_prots, n_prot_variants, n_genomic_variants, sep="\t")

    total_n_genes = df["gene_name"].unique().shape[0]
    total_n_prots = df["prot_acc_version"].unique().shape[0]
    total_n_prot_variants = df["prot_variant"].shape[0]
    total_n_genomic_variants = df[["chrom_num", "chrom_pos", "ref_allele", "alt_allele"]].drop_duplicates(keep="first").shape[0] # these keywords are from ALFAs
    print("total", total_n_genes, total_n_prots, total_n_prot_variants, total_n_genomic_variants, sep="\t")

print_summary(result_df, ["Pathogenic", "Likely-pathogenic"])


	#-genes	#-proteins	#-protein-variants	#-unique-genomic-variants
Pathogenic	967	967	2499	2499
Likely-pathogenic	1207	1210	4804	4803
total	1595	1598	7303	7302
